In [169]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, InputLayer, Dropout

import pandas as pd
import numpy as np
import os
import tensorflow as tf

#DATA NORMALIZATION(STANDARDISATION)
from sklearn.preprocessing import MinMaxScaler

#DATA PLOTTING
import matplotlib.pyplot as plt

#FOR PERFORMANCE METRICS ANALYSIS.
from keras.optimizers import Adam
from keras.losses import  MeanSquaredError
from keras.metrics import  RootMeanSquaredError

#SAVING AND LOADING MODEL
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


In [170]:
nsc_data_frame = pd.read_csv("NSE_data_all_stocks_2022_jan_to_may (1).csv")
nsc_data_frame

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,3-Jan-22,EGAD,Eaagads Ltd,10,15,13.5,13.8,13.5,13.5,-,-,"4,000",-
1,3-Jan-22,KUKZ,Kakuzi Plc,355,427,385,385,385,385,-,-,-,-
2,3-Jan-22,KAPC,Kapchorua Tea Kenya Plc,80,101,99.5,99.5,99.5,95.5,4,4.19%,100,-
3,3-Jan-22,LIMT,Limuru Tea Plc,260,360,320,320,320,320,-,-,-,-
4,3-Jan-22,SASN,Sasini Plc,16.75,22.6,18.7,18.7,18.7,18.7,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727,31-May-22,MSC,Mumias Sugar Company Ltd,0.27,0.27,0.27,0.27,0.27,0.27,-,-,-,-
6728,31-May-22,UNGA,Unga Group Ltd,26.1,36.4,29,29,29,30,-1,3.33%,"2,100",-
6729,31-May-22,SCOM,Safaricom Plc,25.5,45.25,25.95,26.45,26,26.25,-0.25,0.95%,"20,079,900",-
6730,31-May-22,FAHR,Stanlib Fahari Income-REIT,5,7.48,5.5,5.6,5.56,5.58,-0.02,0.36%,"11,700",-


In [171]:
#DATA  CLEANING  # nsc_data_frame.set_index(nsc_data_frame.Date, inplace=True) to update the index  of the dataframe.

date_format = '%d-%b-%y'
nsc_data_frame.Date = pd.to_datetime(nsc_data_frame['Date'], format=date_format)
nsc_data_frame['Day_Of_The_Week'] = nsc_data_frame['Date'].dt.dayofweek

nsc_data_frame['Day_Name'] = nsc_data_frame['Date'].dt.day_name()

nsc_data_frame = nsc_data_frame.drop(columns=['Change', 'Change%', 'Volume', 'Adjusted Price'])


nsc_data_frame['Day_Sin'] = np.sin(2 * np.pi * nsc_data_frame['Day_Of_The_Week']/4.0)
nsc_data_frame['Day_Cos'] = np.cos(2 * np.pi * nsc_data_frame['Day_Of_The_Week']/4.0)

nsc_data_frame.sample(10)



,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Day_Of_The_Week,Day_Name,Day_Sin,Day_Cos
6409,2022-05-25,ABSA,ABSA Bank Kenya Plc,8.94,12.95,9.82,10.05,9.96,9.9,2,Wednesday,1.224647e-16,-1.000000e+00
6370,2022-05-24,CRWN,Crown Paints Kenya Plc,19.05,42.8,37.5,38.2,37.9,38.2,1,Tuesday,1.000000e+00,6.123234e-17
3491,2022-03-16,FTGH,Flame Tree Group Holdings Ltd,1.1,1.45,1.26,1.26,1.26,1.26,2,Wednesday,1.224647e-16,-1.000000e+00
4294,2022-04-04,SASN,Sasini Plc,16.75,22.6,18.6,18.6,18.6,18.6,0,Monday,0.000000e+00,1.000000e+00
99,2022-01-04,BAMB,Bamburi Cement Ltd,33,42,37.5,38,37.75,38.2,1,Tuesday,1.000000e+00,6.123234e-17
2951,2022-03-04,LBTY,Liberty Kenya Holdings Ltd,5.24,10.3,6.62,6.62,6.62,6.98,4,Friday,-2.449294e-16,1.000000e+00
5041,2022-04-21,NBV,Nairobi Business Ventures Ltd,2.68,13.8,4.4,4.48,4.41,4.24,3,Thursday,-1.000000e+00,-1.836970e-16
4393,2022-04-05,KEGN,Kenya Electricity Generating Company Plc,3.49,5.1,3.55,3.6,3.59,3.57,1,Tuesday,1.000000e+00,6.123234e-17
790,2022-01-18,FAHR,Stanlib Fahari Income-REIT,5.5,7.48,6.5,6.5,6.5,6.46,1,Tuesday,1.000000e+00,6.123234e-17
1087,2022-01-25,SCAN,WPP ScanGroup Plc,3.5,6.36,4.19,4.3,4.19,4.19,1,Tuesday,1.000000e+00,6.123234e-17


In [172]:
#EGAD LTD.  DATA

egad_data_frame = nsc_data_frame[nsc_data_frame['Code'] == 'EGAD'].reset_index(drop=True)
egad_data_frame['Day Price'] = pd.to_numeric(egad_data_frame['Day Price'], errors='coerce')


# min_value = egad_data_frame['Day_Of_The_Week'].min()  ------ >> display(egad_data_frame.info())

display(egad_data_frame.head(10))

#SELECT THE 1ST COLUMN AND PLOT THE DAY_SINE AGAINST TIME

# plt.figure(figsize=(12, 6))  
# plt.subplot(1, 2, 1)  # Selects the first subplot
# egad_data_frame['Day_Sin'].plot()
# plt.title('Day_Sin Over Time')

#SELECT THE 2ND COLUMN AND PLOT THE DAY PRICE AGAINST TIME

# plt.subplot(1, 2, 2)  
# egad_data_frame['Day Price'].plot()
# plt.title('Day Price Over Time')
# 
# plt.tight_layout()  # To improve subplot spacing
# plt.show()




,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Day_Of_The_Week,Day_Name,Day_Sin,Day_Cos
0,2022-01-03,EGAD,Eaagads Ltd,10,15,13.5,13.8,13.50,13.5,0,Monday,0.000000e+00,1.000000e+00
1,2022-01-04,EGAD,Eaagads Ltd,10,15,12.85,12.85,12.85,13.5,1,Tuesday,1.000000e+00,6.123234e-17
2,2022-01-05,EGAD,Eaagads Ltd,10,15,12.85,12.85,12.85,12.85,2,Wednesday,1.224647e-16,-1.000000e+00
3,2022-01-06,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,12.85,3,Thursday,-1.000000e+00,-1.836970e-16
4,2022-01-07,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,13.8,4,Friday,-2.449294e-16,1.000000e+00
5,2022-01-10,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,13.8,0,Monday,0.000000e+00,1.000000e+00
6,2022-01-11,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,13.8,1,Tuesday,1.000000e+00,6.123234e-17
7,2022-01-12,EGAD,Eaagads Ltd,10,15,12.9,12.9,12.90,13.8,2,Wednesday,1.224647e-16,-1.000000e+00
8,2022-01-13,EGAD,Eaagads Ltd,10,15,12.9,12.9,12.90,12.9,3,Thursday,-1.000000e+00,-1.836970e-16
9,2022-01-14,EGAD,Eaagads Ltd,10,15,13.8,13.8,13.80,12.9,4,Friday,-2.449294e-16,1.000000e+00


In [173]:
df_interest = egad_data_frame[['Day Price', 'Day_Sin']].copy()

df_interest

,Day Price,Day_Sin
0,13.50,0.000000e+00
1,12.85,1.000000e+00
2,12.85,1.224647e-16
3,13.80,-1.000000e+00
4,13.80,-2.449294e-16
...,...,...
97,13.45,1.224647e-16
98,13.45,-1.000000e+00
99,13.45,-2.449294e-16
100,13.45,0.000000e+00


In [186]:
#Splitting the Dataframe into X_features and the Y_Classes
"""
WINDOW_SIZE = 5
[
  [[ds1, dp1], [ds2, dp2], [ds3, dp3], [ds4, dp4], [ds5, dp5]],
  [[ds2, dp2], [ds3, dp3], [ds4, dp4], [ds5, dp5], [ds6, dp6]]  
]
"""

def __generate_x_y_from_nsc_df_method_1(__data_frame , time_step=5):
    data_array = __data_frame.to_numpy()
    X = []
    y = []
    for i in range(len(data_array)-time_step):
        
        rows = data_array[i:i + time_step]   # print(rowing[1]) gives [12.85  1.] , that is the second row 
        
        zipped_row = [row for row in rows]  # creates a list of nd_arrays .
        
        label = data_array[i + time_step][0]
        X.append(zipped_row)
        y.append(label)
        
    return np.array(X), np.array(y)

X_data, Y_data = __generate_x_y_from_nsc_df_method_1(df_interest , 5)

print("X_Data shape : ",X_data.shape)

print(X_data.shape , Y_data.shape)

X_data[0]



X_Data shape :  (97, 5, 2)
(97, 5, 2) (97,)


array([[ 1.3500000e+01,  0.0000000e+00],
       [ 1.2850000e+01,  1.0000000e+00],
       [ 1.2850000e+01,  1.2246468e-16],
       [ 1.3800000e+01, -1.0000000e+00],
       [ 1.3800000e+01, -2.4492936e-16]])

In [176]:
#Model Creation and Model Compilation.
dropout_rate = 0.2
input_shape=(5, 6)

model = Sequential([Input(input_shape),
                    LSTM(units=100, return_sequences=True),
                    Dropout(rate=dropout_rate),
                    LSTM(units=100, return_sequences=False),
                    Dropout(rate= dropout_rate),
                    Dense(units=32, activation='relu'),
                    Dense(units = 1)])

model.compile(loss=MeanSquaredError(), 
              optimizer=Adam(learning_rate=0.0001),
              metrics=[RootMeanSquaredError()])
